In [3]:
import numpy as np
import scipy.sparse as sps
import scipy as sp
from matplotlib import pyplot as plt

import line_profiler
%load_ext line_profiler

In [4]:
# create numpy array
R = np.array(
    [
        [1, -1, 1, -1, 1, -1],
        [1, 1, 0, -1, -1, -1],
        [0, 1, 1, -1, -1, 0],
        [-1, -1, -1, 1, 1, 1],
        [-1, 0, -1, 1, 1, 1],
    ],
    dtype=float,
)
ij_missing = R == 0

In [300]:
#
num_users = 5
num_items = 6
num_movies = num_items
Rs = sps.coo_array(R)
Rsc = Rs.tocsc()
Rsr = Rs.tocsr()

#
nnz_entries_per_col = np.diff(Rsc.indptr)
nnz_entries_per_row = np.diff(Rsr.indptr)

nonzero_rows = Rsc.nonzero()[0]
nonzero_cols = Rsc.nonzero()[1]

# find mean of each row and column
meanCol = Rsc.mean(axis=0) * num_users / nnz_entries_per_col
meanRow = Rsc.mean(axis=1) * num_items / nnz_entries_per_row

In [7]:
# implement svd algorithm using scipy.sparse.svd
# This approach requires a dense matrix since each iteration
# fills in the missing values with the latest prediction
# Can avoid dense matrix if only one iteration is performed and
# no initial guess is given, but that will likely not converge
#

# first fill in missing elements with row (user) mean
# Note: for this example, the mean centering did not
# affect prediction but led to faster convergence
R_mean = R.copy()
R_mean[1, 2] = meanRow[1]
R_mean[2, 0] = meanRow[2]
R_mean[2, 5] = meanRow[2]
R_mean[4, 1] = meanRow[4]

# iterate until convergence:
for j in range(10):
    # construct sparse matrix
    Rsc = sps.coo_array(R_mean).tocsc()

    # next run svd on the filled in matrix
    U, sig, V = sps.linalg.svds(Rsc, k=2)

    # compute new matrix
    resulting_matrix = U @ np.diag(sig) @ V

    # reset known ratings in resulting matrix
    R_mean[ij_missing] = resulting_matrix[ij_missing]

    print(np.round(resulting_matrix[ij_missing], 4))
    print()

[ 0.5881  0.43   -0.43   -0.2095]

[ 0.8826  0.6677 -0.6677 -0.5092]

[ 0.9873  0.8008 -0.8008 -0.6995]

[ 1.0188  0.8772 -0.8772 -0.8156]

[ 1.0236  0.9226 -0.9226 -0.8862]

[ 1.0201  0.9503 -0.9503 -0.9295]

[ 1.0151  0.9678 -0.9678 -0.9562]

[ 1.0107  0.979  -0.979  -0.9727]

[ 1.0073  0.9862 -0.9862 -0.983 ]

[ 1.0049  0.9909 -0.9909 -0.9894]



In [310]:
# given matrix
R = np.array(
    [
        [1, -1, 1, -1, 1, -1],
        [1, 1, 0, -1, -1, -1],
        [0, 1, 1, -1, -1, 0],
        [-1, -1, -1, 1, 1, 1],
        [-1, 0, -1, 1, 1, 1],
    ],
    dtype=float,
)

Rs = sps.coo_array(R)
Rsc = Rs.tocsc()
Rsr = Rs.tocsr()

nnz_row_per_col = np.diff(Rsc.indptr)
nnz_col_per_row = np.diff(Rsr.indptr)

nonzero_rows = Rsc.nonzero()[0]
nonzero_cols = Rsc.nonzero()[1]

# find mean of each row and column
meanCol = Rsc.mean(axis=0)*num_users/nnz_row_per_col
meanRow = Rsc.mean(axis=1)*num_items/nnz_col_per_row

# find std of each row and column
Rsc2 = Rsc**2
stdCol = np.sqrt(Rsc2.mean(axis=0)*num_users/nnz_row_per_col - (Rsc.mean(axis=0)*num_users/nnz_row_per_col)**2)
stdRow = np.sqrt(Rsc2.mean(axis=1)*num_items/nnz_col_per_row - (Rsc.mean(axis=1)*num_items/nnz_col_per_row)**2)

# compute alternate arrays
Rsc_mean = Rsc.copy()
Rsc_mean.data = Rsc_mean.data - np.take(meanRow, Rsc_mean.indices)

Rsr_mean = Rsc_mean.tocsr()



# compute set of observed indices (i, j) = (rows, cols)
irows, jcols, vals = sps.find(Rsc)
num_nonzero_entries = len(vals)


In [311]:
meanRow

array([ 0.00000000e+00, -2.00000000e-01,  0.00000000e+00, -5.55111512e-17,
        2.00000000e-01])

In [308]:
def svd():
    # implement matrix factorization with regularization
    # use stochastic gradient descent for updates


    k = 2  # number of latent factors
    nbatch = 2
    alpha = 0.005*num_nonzero_entries/nbatch  # learning rate
    lam = 0.0  # regularization parameter
    # inialize arrays

    # U and V are full numpy arrays (dense)
    U = np.random.random(size=(num_users, k)) * 2 - 1  # random numbers between [-1,1]
    V = np.random.random(size=(num_movies, k)) * 2 - 1  # random numbers between [-1,1]
    U_final = np.zeros(shape=U.shape)
    V_final = np.zeros(shape=V.shape)

    # E is sparse error matrix
    E = Rsc_mean.copy()
    Et = Rsr_mean.copy()

    tole = 1e-3
    n_epochs = 1


    Etemp = Rsc_mean.copy()
    Etempt = Rsr_mean.copy()
    for k_epoch in range(n_epochs):
        u_old = 0
        v_old = 0
        for q in range(k):
            u_new = np.dot(U[:, q], U[:, q])
            v_new = np.dot(V[:, q], V[:, q])
            num_iter = 0
            while np.abs(u_new - u_old) > tole and np.abs(v_new - v_old) > tole:
                num_iter += 1
                u_old = u_new
                v_old = v_new
                U_old = U.copy()
                V_old = V.copy()

                random_shuffle = np.arange(num_nonzero_entries)
                # random_shuffle = np.unique(np.random.randint(0,num_nonzero_entries,size=(nbatch,)))
                np.random.shuffle(random_shuffle)  # shuffle in place

                # Etemp.data = np.array([0]*len(Etemp.data))
                # Etempt.data = np.array([0]*len(Etempt.data))

                for ij in random_shuffle:
                    i = irows[ij]
                    j = jcols[ij]

                    temp2 = np.dot(U_old[i, :], V_old[j, :])

                    # using csc format to update xx
                    ind_j0 = E.indptr[j]
                    ind_j1 = E.indptr[j + 1]
                    e_ind = E.indices[ind_j0:ind_j1]
                    Etemp.data[ind_j0:ind_j1][e_ind == i] = temp2

                    # using csr format to update xx
                    ind_j0 = Et.indptr[i]
                    ind_j1 = Et.indptr[i + 1]
                    et_ind = Et.indices[ind_j0:ind_j1]
                    Etempt.data[ind_j0:ind_j1][et_ind == j] = temp2

                E.data = Rsc_mean.data - Etemp.data
                Et.data = Rsr_mean.data - Etempt.data

                # consider larger batches and doing matrix multiplication
                for ij in random_shuffle:
                    i = irows[ij]
                    j = jcols[ij]

                    # using csc format to update xx
                    ind_j0 = E.indptr[j]
                    ind_j1 = E.indptr[j + 1]
                    e_ind = E.indices[ind_j0:ind_j1]
                    e_dat = E.data[ind_j0:ind_j1]

                    # using csr format to update xx
                    ind_j0 = Et.indptr[i]
                    ind_j1 = Et.indptr[i + 1]
                    et_ind = Et.indices[ind_j0:ind_j1]
                    et_dat = Et.data[ind_j0:ind_j1]

                    U[i, q] = U_old[i, q] + alpha *  (
                        np.dot(et_dat, V_old[et_ind, q]) - lam * U_old[i, q]
                    )
                    V[j, q] = V_old[j, q] + alpha * (
                        np.dot(e_dat, U_old[e_ind, q]) - lam * V_old[j, q]
                    )
                u_new = np.dot(U[:, q], U[:, q])
                v_new = np.dot(V[:, q], V[:, q])
            # R2 = R2 - U[:,[q]] @ np.transpose(V[:,[q]])
        U_final = U_final + U
        V_final = V_final + V

    U_final = U_final / n_epochs
    V_final = V_final / n_epochs

    Rpred = U_final @ V_final.T
    for ii in range(5):
        Rpred[:, ii] = Rpred[:, ii] + meanRow[ii]
    # Rpred[Rpred > 0] = 1
    # Rpred[Rpred < 0] = -1
    print(Rpred[ij_missing])

In [309]:
%lprun -f svd svd()

[ 1.19563305  1.06272351 -1.04146598 -0.84639851]


Timer unit: 1e-07 s

Total time: 0.0334904 s
File: C:\Users\Nick\AppData\Local\Temp\ipykernel_13948\446836788.py
Function: svd at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def svd():
     2                                               # implement matrix factorization with regularization
     3                                               # use stochastic gradient descent for updates
     4                                           
     5                                           
     6         1          9.0      9.0      0.0      k = 2  # number of latent factors
     7         1          3.0      3.0      0.0      nbatch = 2
     8         1         17.0     17.0      0.0      alpha = 0.005*num_nonzero_entries/nbatch  # learning rate
     9         1          3.0      3.0      0.0      lam = 0.0  # regularization parameter
    10                                               # inialize arrays
    11    

In [269]:
# random_shuffle = np.arange(len(jcols))
# np.random.shuffle(random_shuffle)  # shuffle in place

# for ij in random_shuffle[0:5]:
# i = irows[ij]
# j = jcols[ij]

i = irows[random_shuffle[0:5]]
j = jcols[random_shuffle[0:5]]
temp2 = np.sum(U_old[i, :]*V_old[j, :],axis=1)

# using csc format to update xx
ind_j0 = E.indptr[j]
ind_j1 = E.indptr[j + 1]
e_ind = E.indices[ind_j0:ind_j1]
Etemp.data[ind_j0:ind_j1][e_ind == i] = temp2

# # using csr format to update xx
# ind_j0 = Et.indptr[i]
# ind_j1 = Et.indptr[i + 1]
# et_ind = Et.indices[ind_j0:ind_j1]
# Etempt.data[ind_j0:ind_j1][et_ind == j] = temp2

# E.data = Rsc_mean.data - Etemp.data
# Et.data = Rsr_mean.data - Etempt.data

TypeError: only integer scalar arrays can be converted to a scalar index

In [288]:
ind_j0 = E.indptr[j]
ind_j1 = E.indptr[j + 1]
print(ind_j0)
print(ind_j1)

e_ind = E.indices[np.r_[tuple(slice(s, e) for s, e in zip(ind_j0,ind_j1))]]

[17  0 12 17  8]
[22  4 17 22 12]


In [287]:
Etemp.data[ind_j0:ind_j1][e_ind == i] = temp2

array([0, 1, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 2, 3,
       4])

In [292]:
Etemp.data[np.r_[tuple(slice(s, e) for s, e in zip(ind_j0,ind_j1))]]

array([ 0.34729925, -1.01195332, -0.94655799,  0.9668216 ,  0.38105861,
        0.75576971,  1.10648956, -1.15245143, -1.22819195, -0.79312914,
       -0.80052277, -0.99253231,  0.85586837,  1.06158417,  0.34729925,
       -1.01195332, -0.94655799,  0.9668216 ,  0.38105861,  0.85449777,
        1.146598  , -0.9960005 , -1.19258012])

In [293]:
e_ind

array([0, 1, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 2, 3,
       4])

In [294]:
i


array([0, 1, 3, 4, 2])

array([0, 1, 3, 4, 5])

In [189]:
0.00018366039756939623

temp = Rsc.copy()
# temp.data = np.zeros(shape=temp.data.shape)

# temp.data
temp.toarray()

array([[ 1., -1.,  1., -1.,  1., -1.],
       [ 1.,  1.,  0., -1., -1., -1.],
       [ 0.,  1.,  1., -1., -1.,  0.],
       [-1., -1., -1.,  1.,  1.,  1.],
       [-1.,  0., -1.,  1.,  1.,  1.]])

In [217]:
def index_sparse_col_array(R,row,col):
    # returns array([], dtype=float64) if R[row,col] is missing 
    ind_j0 = R.indptr[col]
    ind_j1 = R.indptr[col+1]
    bb = R.indices[ind_j0:ind_j1]
    cc = R.data[ind_j0:ind_j1]     
    return cc[bb == row]

In [218]:
index_sparse_col_array(Rsc,0,0)

array([1.])

In [223]:
ind_j0 = Rsc.indptr[0]
ind_j1 = Rsc.indptr[0+1]
bb = Rsc.indices[ind_j0:ind_j1]
Rsc.data[ind_j0:ind_j1][bb == 0] = 5

In [224]:
Rsc.toarray()

array([[ 5., -1.,  1., -1.,  1., -1.],
       [ 2.,  1.,  0., -1., -1., -1.],
       [ 0.,  1.,  1., -1., -1.,  0.],
       [ 2., -1., -1.,  1.,  1.,  1.],
       [ 2.,  0., -1.,  1.,  1.,  1.]])

# Implement SVD using SurPRISE library

In [115]:
import pandas as pd

from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {
    "userID": [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4],
    "itemID": [0, 1, 2, 3, 4, 5, 0, 1, 3, 4, 5, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 0, 2, 3, 4, 5],
    "rating": [1,-1, 1,-1, 1,-1, 1, 1,-1,-1,-1, 1, 1,-1,-1,-1,-1,-1, 1, 1, 1,-1,-1, 1, 1, 1],
}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(-1, 1))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
# cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
trainset = data.build_full_trainset()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
predictions